In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [11]:
torch.manual_seed(42)

In [12]:
# GPU Checking
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [13]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [14]:
train_dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

In [15]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [16]:
# df=pd.read_csv("data/fmnist_small.csv")
# df.head()

In [17]:
# plt.imshow(df.iloc[0:1,1:].values.reshape(28,28))

In [18]:
# X=df.iloc[:,1:].values
# y=df.iloc[:,0].values

In [19]:
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [20]:
# X_train=X_train/255.0
# X_test=X_test/255.0

In [21]:
# # features->float32  labels->long
# class CustomDataset(Dataset):
#     def __init__(self,features,labels):
#         self.features=torch.tensor(features,dtype=torch.float32)
#         self.labels=torch.tensor(labels,dtype=torch.long)
    
#     def __len__(self):
#         return len(self.features)
    
#     def __getitem__(self,idx):
#         return self.features[idx],self.labels[idx]

In [22]:
# train_dataset= CustomDataset(X_train,y_train)
# test_dataset=CustomDataset(X_test,y_test)

In [23]:
# train_dataset[0]

In [24]:
# # testing shuffle off to compute better accuracy
# train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True, pin_memory=True)
# test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False, pin_memory=True)

In [30]:
class MyNN(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.model=nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_features,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            # dont define softmax as it is inbuilt in the BCE loss
            nn.Linear(64,10)
        )
    
    def forward(self,x):
        return self.model(x)

In [31]:
epochs=100
lr=0.1

In [32]:
model=MyNN(28*28)

# -------------------------------
model=model.to(device)
# -------------------------------

criterion=nn.CrossEntropyLoss()

optimizer=optim.SGD(model.parameters(),lr, weight_decay=1e-4)

In [33]:
for epoch in range(epochs):

    epochLoss=0

    for batch_features, batch_labels in train_loader:


        # -------------------------------
        batch_features=batch_features.to(device)
        batch_labels=batch_labels.to(device)
        # -------------------------------


        # forward pass
        outputs=model(batch_features)

        # loss
        loss=criterion(outputs,batch_labels)

        # back pass
        optimizer.zero_grad()
        loss.backward()

        # update grads
        optimizer.step()

        epochLoss+=loss.item()
    
    if((epoch+1)%10==0):
        avgLoss= epochLoss/len(train_loader)
        print(f'Epoch: {epoch+1}, Loss:{avgLoss}')

KeyboardInterrupt: 

In [ ]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
total=0
correct=0

with torch.no_grad():

    for batch_features, batch_labels in test_loader:

        # -------------------------------
        batch_features=batch_features.to(device)
        batch_labels=batch_labels.to(device)
        # -------------------------------



        # batch size=32, every input gives 10 len tensor with probabilities
        # so we get (32,10) len tensor
        pred=model(batch_features)
        _, predicted= torch.max(pred,1)

        total=total+batch_labels.shape[0]
        correct=correct+(predicted==batch_labels).sum().item()

print(f'Accuracy: {correct/total}')

Accuracy: 0.8391666666666666
